In [325]:
import sqlite3
import pandas as pd

con = sqlite3.connect("database.sqlite")

In [326]:
# country_id="21518" and league_id="21518" define La Liga
# id="43040" or team_api_id="8633" define Real Madrid
# id="43039" or team_api_id="9783" define RC Deportivo de La Coruña (opponent of Real Madrid in stage 38)
df = pd.read_sql_query('SELECT stage, date, home_team_goal, away_team_goal, '
                       'home_team_api_id, away_team_api_id, '
                       'B365H, B365D, B365A '
                       'FROM Match '
                       'WHERE country_id="21518" '
                       'AND league_id="21518" '
                       'ORDER BY season, stage ASC', con)

In [327]:
team = 8633

In [328]:
teams = df['home_team_api_id'].unique()
points_against_teams = {}
points_of_team = {}
clashes = {}
appearances = {}
for team_id in teams:
    points_against_teams[team_id] = {}
    clashes[team_id] = {}
    points_of_team[team_id] = []
    appearances[team_id] = 0
    for team_id_inner in teams:
        if team_id != team_id_inner:
            points_against_teams[team_id][team_id_inner] = []
            clashes[team_id][team_id_inner] = 0
for i, value in enumerate(df.values):
    home_team_api_id = df['home_team_api_id'].values[i]
    away_team_api_id = df['away_team_api_id'].values[i]
    home_team_goal = df['home_team_goal'].values[i]
    away_team_goal = df['away_team_goal'].values[i]
    if home_team_goal > away_team_goal:
        points_against_teams[home_team_api_id][away_team_api_id].append(3)
        points_against_teams[away_team_api_id][home_team_api_id].append(0)
        points_of_team[home_team_api_id].append(3)
        points_of_team[away_team_api_id].append(0)
    elif home_team_goal < away_team_goal:
        points_against_teams[home_team_api_id][away_team_api_id].append(0)
        points_against_teams[away_team_api_id][home_team_api_id].append(3)
        points_of_team[home_team_api_id].append(0)
        points_of_team[away_team_api_id].append(3)
    else:
        points_against_teams[home_team_api_id][away_team_api_id].append(1)
        points_against_teams[away_team_api_id][home_team_api_id].append(1)
        points_of_team[home_team_api_id].append(1)
        points_of_team[away_team_api_id].append(1)


In [329]:
bets_of_team = []
enemies_of_team = []
wins_of_team_against_enemy_in_last_five = []
wins_of_team_in_last_five = []
points_of_team_in_last_five = []
for i, value in enumerate(df.values):
    home_team_api_id = df['home_team_api_id'].values[i]
    away_team_api_id = df['away_team_api_id'].values[i]
    if home_team_api_id == team:
        enemies_of_team.append(df['away_team_api_id'].values[i])
        bets_of_team.append(df['B365H'].values[i])
        wins_against = 0
        wins = 0
        points = 0
        for j in range(clashes[home_team_api_id][away_team_api_id] - 5, clashes[home_team_api_id][away_team_api_id]):
            if j >= 0 and points_against_teams[home_team_api_id][away_team_api_id][j] == 3:
                wins_against += 1
        for j in range(appearances[home_team_api_id] - 5, appearances[home_team_api_id]):
            if j >= 0 and points_of_team[home_team_api_id][j] == 3:
                wins += 1
            points += points_of_team[home_team_api_id][j]
        wins_of_team_against_enemy_in_last_five.append(wins_against)
        wins_of_team_in_last_five.append(wins)
        points_of_team_in_last_five.append(points)
        clashes[home_team_api_id][away_team_api_id] += 1
        clashes[away_team_api_id][home_team_api_id] += 1
        appearances[home_team_api_id] += 1
    if away_team_api_id == team:
        enemies_of_team.append(df['home_team_api_id'].values[i])
        bets_of_team.append(df['B365A'].values[i])
        wins_against = 0
        wins = 0
        points = 0
        for j in range(clashes[away_team_api_id][home_team_api_id] - 5, clashes[away_team_api_id][home_team_api_id]):
            if j >= 0 and points_against_teams[away_team_api_id][home_team_api_id][j] == 3:
                wins_against += 1
        for j in range(appearances[away_team_api_id] - 5, appearances[away_team_api_id]):
            if j >= 0 and points_of_team[away_team_api_id][j] == 3:
                wins += 1
            points += points_of_team[away_team_api_id][j]
        wins_of_team_against_enemy_in_last_five.append(wins_against)
        wins_of_team_in_last_five.append(wins)
        points_of_team_in_last_five.append(points)
        clashes[home_team_api_id][away_team_api_id] += 1
        clashes[away_team_api_id][home_team_api_id] += 1
        appearances[away_team_api_id] += 1

In [330]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch
import torch.nn as nn
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive
from tqdm.auto import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

In [331]:
class BayesianRegression(PyroModule):
    def __init__(self, input_features=2, h1=20, h2=20):
        super().__init__()
        self.input_features=input_features
        self.fc1 = PyroModule[nn.Linear](input_features, h1)
        self.fc1.weight = PyroSample(dist.Normal(0., 1.).expand([h1, input_features]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(0., 1.).expand([h1]).to_event(1))
        self.fc2 = PyroModule[nn.Linear](h1, h2)
        self.fc2.weight = PyroSample(dist.Normal(0., 1.).expand([h2, h1]).to_event(2))
        self.fc2.bias = PyroSample(dist.Normal(0., 1.).expand([h2]).to_event(1))
        self.fc3 = PyroModule[nn.Linear](h2, 1)
        self.fc3.weight = PyroSample(dist.Normal(0., 1.).expand([1, h2]).to_event(2))
        self.fc3.bias = PyroSample(dist.Normal(0., 1.).expand([1]).to_event(1))
        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        x = x.reshape(-1, self.input_features)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        mu = self.fc3(x).squeeze()
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.))
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma), obs=y)
        return mu

In [332]:
model = BayesianRegression()
guide = AutoDiagonalNormal(model)
adam = pyro.optim.Adam({"lr": 1e-3})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [333]:
x_data = torch.tensor([np.array(wins_of_team_in_last_five[50:250]), np.array(wins_of_team_against_enemy_in_last_five[50:250])], dtype=torch.float)
x_data_test = torch.tensor([np.array(wins_of_team_in_last_five[250:304]), np.array(wins_of_team_against_enemy_in_last_five[250:304])], dtype=torch.float)
y_data = torch.from_numpy(np.array(bets_of_team[50:250])).float()


In [334]:
num_iterations = 2000
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_data)))

[iteration 0001] loss: 888.9071
[iteration 0101] loss: 555.0861
[iteration 0201] loss: 619.3645
[iteration 0301] loss: 615.0531
[iteration 0401] loss: 521.4004
[iteration 0501] loss: 507.6635
[iteration 0601] loss: 475.7505
[iteration 0701] loss: 464.4979
[iteration 0801] loss: 483.6058
[iteration 0901] loss: 641.2886
[iteration 1001] loss: 439.4358
[iteration 1101] loss: 421.2929
[iteration 1201] loss: 441.3437
[iteration 1301] loss: 390.4239
[iteration 1401] loss: 382.7052
[iteration 1501] loss: 377.7707
[iteration 1601] loss: 363.7083
[iteration 1701] loss: 340.2423
[iteration 1801] loss: 362.3267
[iteration 1901] loss: 398.6936


In [335]:
predictive = Predictive(model, guide=guide, num_samples=500)
preds = predictive(x_data_test)

y_pred = preds['obs'].T.detach().numpy().mean(axis=1)
y_std = preds['obs'].T.detach().numpy().std(axis=1)

In [336]:
error = 0
for i, bet in enumerate(y_pred):
    error += abs(bet - bets_of_team[250 + i])
avg_error = error / 54
print(avg_error)


0.39713122160346426
